# Skin Disease classification using CNN Algorithm

# Import Libraries

In [1]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os

In [2]:
IMG_SAVE_PATH = r'train' 

In [3]:
Str_to_Int = {
    'Actinic keratosis': 0,
    'Atopic Dermatitis': 1,
    'Benign keratosis': 2,
    'Dermatofibroma': 3,
    'Melanocytic nevus': 4,
    'Melanoma':5,
    'Squamous cell carcinoma': 6,
    'Tinea Ringworm Candidiasis':7,
    'Vascular lesion': 8
}

NUM_CLASSES = 9


def str_to_Int_mapper(val):
    return Str_to_Int[val]

# Data Preprocessing


In [4]:
import PIL
import cv2 
from PIL import Image
dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    for image in os.listdir(path):
        new_path = os.path.join(path, image)
        try:
            imgpath=PIL.Image.open(new_path)
            #if imgpath.shape!=(240,240,3):
            imgpath=imgpath.convert('RGB')
            img = np.asarray(imgpath)
            img = cv2.resize(img, (240,240))
            img=img/255.
            dataset.append([img, directory]) 
        except FileNotFoundError:
            print('Image file not found. Skipping...')

In [5]:
data, labels = zip(*dataset)
temp = list(map(str_to_Int_mapper, labels))

In [6]:
len(data)

845

In [7]:
#data[0]

In [8]:
import keras
labels = keras.utils.to_categorical(temp)

In [9]:
count=0
for i in data:
    if i.shape!=(240, 240, 3):
        count=count+1
print(count)      

0


# CNN

In [10]:
def cnn():
    model=Sequential()

    model.add(Conv2D(8, kernel_size=(3,3), activation='relu', input_shape=(240,240,3)))
    model.add(Conv2D(16, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))
    
    return model

In [11]:
def split_data(X, y, train_size=0.7, val_size=0.15):
    total_size = len(X)
    
    train_end = int(train_size * total_size)
    val_end = train_end + int(val_size * total_size)
    
    X_train = X[:train_end]
    y_train = y[:train_end]
    
    X_val = X[train_end:val_end]
    y_val = y[train_end:val_end]
    
    X_test = X[val_end:]
    y_test = y[val_end:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = split_data(np.array(data), np.array(labels))

In [33]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5, random_state=42)


In [ ]:
import optuna
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D, Flatten

def objective(trial):
    model = Sequential()
    
    kernel_size = trial.suggest_int('kernel_size', 3, 8)    
    
    model.add(Conv2D(trial.suggest_int('conv1_units', 4, 32), kernel_size=(kernel_size, kernel_size), activation='relu', input_shape=(240, 240, 3)))
    model.add(Conv2D(trial.suggest_int('conv2_units', 4, 32), kernel_size=(kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(trial.suggest_uniform('dropout1', 0.5, 1)))
              
    model.add(Conv2D(trial.suggest_int('conv3_units', 4, 32), kernel_size=(kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(trial.suggest_int('conv4_units', 4, 32), kernel_size=(kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(trial.suggest_uniform('dropout2', 0.5, 1)))
              
    model.add(Flatten())
    model.add(Dense(trial.suggest_int('dense_units', 32, 128), activation='relu'))
    model.add(Dropout(trial.suggest_uniform('dropout3', 0.5, 1)))
    model.add(Dense(9, activation='softmax'))
    
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=64)
    
    return history.history['val_accuracy'][-1]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

best_params = study.best_params

c:\Users\johnk\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-03 20:39:30,807] A new study created in memory with name: no-name-35aac7a6-a54b-4397-a9e6-3bdd51c5c019
C:\Users\johnk\AppData\Local\Temp\ipykernel_20096\4021640977.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  model.add(Dropout(trial.suggest_uniform('dropout1', 0.5, 1)))
C:\Users\johnk\AppData\Local\Temp\ipykernel_20096\4021640977.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  model.add(Dropout(trial.suggest_

Epoch 1/5
10/10 [==============================] - 77s 8s/step - loss: 594.3774 - accuracy: 0.1540 - val_loss: 2.9696 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 75s 7s/step - loss: 1.9466 - accuracy: 0.1624 - val_loss: 3.6258 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 74s 7s/step - loss: 1.8888 - accuracy: 0.1726 - val_loss: 4.0743 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 44s 4s/step - loss: 1.8731 - accuracy: 0.1692 - val_loss: 4.3086 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 42s 4s/step - loss: 1.8679 - accuracy: 0.1658 - val_loss: 4.3851 - val_accuracy: 0.0000e+00


[I 2023-11-03 20:44:43,808] Trial 0 finished with value: 0.0 and parameters: {'kernel_size': 6, 'conv1_units': 16, 'conv2_units': 27, 'dropout1': 0.7767879553070001, 'conv3_units': 8, 'conv4_units': 24, 'dropout2': 0.6339202185210618, 'dense_units': 68, 'dropout3': 0.9320201689561083, 'learning_rate': 0.06987226366249664}. Best is trial 0 with value: 0.0.


Epoch 1/5
10/10 [==============================] - 35s 3s/step - loss: 13373.8652 - accuracy: 0.1455 - val_loss: 3.6955 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 32s 3s/step - loss: 1.9350 - accuracy: 0.1726 - val_loss: 4.1754 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 32s 3s/step - loss: 1.8762 - accuracy: 0.1760 - val_loss: 4.3715 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 35s 4s/step - loss: 1.8655 - accuracy: 0.1472 - val_loss: 4.5212 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 33s 3s/step - loss: 1.8572 - accuracy: 0.1692 - val_loss: 4.6782 - val_accuracy: 0.0000e+00


[I 2023-11-03 20:47:32,623] Trial 1 finished with value: 0.0 and parameters: {'kernel_size': 4, 'conv1_units': 7, 'conv2_units': 29, 'dropout1': 0.9055192726652057, 'conv3_units': 23, 'conv4_units': 5, 'dropout2': 0.6958552955512065, 'dense_units': 66, 'dropout3': 0.5988626810058928, 'learning_rate': 0.13650671838528017}. Best is trial 0 with value: 0.0.


Epoch 1/5
10/10 [==============================] - 61s 6s/step - loss: 172729.2031 - accuracy: 0.1404 - val_loss: 3.6848 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 58s 6s/step - loss: 5.7712 - accuracy: 0.1523 - val_loss: 4.0458 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 41s 4s/step - loss: 1.8892 - accuracy: 0.1421 - val_loss: 4.2033 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 56s 5s/step - loss: 1.8683 - accuracy: 0.1692 - val_loss: 4.5922 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 24s 2s/step - loss: 1.8598 - accuracy: 0.1523 - val_loss: 4.7991 - val_accuracy: 0.0000e+00


[I 2023-11-03 20:51:33,051] Trial 2 finished with value: 0.0 and parameters: {'kernel_size': 6, 'conv1_units': 18, 'conv2_units': 16, 'dropout1': 0.8685856735810668, 'conv3_units': 16, 'conv4_units': 14, 'dropout2': 0.6457689696892548, 'dense_units': 111, 'dropout3': 0.7436134228848712, 'learning_rate': 0.12738038811974076}. Best is trial 0 with value: 0.0.


In [11]:
from itertools import chain
X_train.shape
flattened = X_train.reshape(X_train.shape[0], -1)

len(flattened)


NameError: name 'X_train' is not defined

In [21]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [34]:
import xgboost as xgb
features = np.array(X_train)
features = features.reshape(features.shape[0], -1)
#.reshape(X_train.shape[0], -1)

# Create an XGBoost DMatrix from your features and labels
dtrain = xgb.DMatrix(data=features, label=y_train)

# Set up XGBoost parameters
params = {
    #'reg_lambda': 3000.0,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',  # Use an appropriate evaluation metric
}

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=1)

# Make predictions
predictions = model.predict(dtrain)

XGBoostError: [22:02:50] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\io.h:232: bad_malloc: Failed to allocate 9306676944 bytes.

In [27]:
from sklearn.metrics import accuracy_score
argmax_predictions = []
argmaxy=[]
for sublist in predictions:
    argmax = np.argmax(sublist)
    argmax_predictions.append(argmax)
for sublist in y_train:
    argmax = np.argmax(sublist)
    argmaxy.append(argmax)
accuracy_score(argmaxy, argmax_predictions)




ValueError: Found input variables with inconsistent numbers of samples: [591, 128]

In [21]:
features = X_test.reshape(X_test.shape[0], -1)
dtest = xgb.DMatrix(data=features, label=y_test)
predictions = model.predict(dtest)
argmax_predictions = []
argmaxy=[]
for sublist in predictions:
    argmax = np.argmax(sublist)
    argmax_predictions.append(argmax)
for sublist in y_test:
    argmax = np.argmax(sublist)
    argmaxy.append(argmax)
accuracy_score(argmaxy, argmax_predictions)

0.0

In [25]:
np.unique(argmax_predictions, return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6], dtype=int64),
 array([44, 11,  5, 47, 11,  9,  1], dtype=int64))

In [35]:
len(X_train)

422

In [13]:
def train_and_record_accuracy(model, train_images, train_labels, test_images, test_labels, num_epochs):
    history = {"train_accuracy": [], "val_accuracy": []}
    for epoch in range(num_epochs):
        model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels), batch_size=64)
        train_accuracy = model.evaluate(train_images, train_labels, verbose=0)[1]
        val_accuracy = model.evaluate(test_images, test_labels, verbose=0)[1]
        history["train_accuracy"].append(train_accuracy)
        history["val_accuracy"].append(val_accuracy)
        print(f"Epoch {epoch + 1}: Train Accuracy = {train_accuracy:.4f}, Validation Accuracy = {val_accuracy:.4f}")
    return history

In [14]:
import pandas as pd
post_search_df = study.trials_dataframe()
post_search_df

,number,value,datetime_start,datetime_complete,duration,params_conv1_units,params_conv2_units,params_conv3_units,params_conv4_units,params_dense_units,params_dropout1,params_dropout2,params_dropout3,params_kernel_size,params_learning_rate,state
0,0,0.0,2023-11-03 20:39:30.807939,2023-11-03 20:44:43.807108,0 days 00:05:12.999169,16,27,8,24,68,0.776788,0.633920,0.932020,6,0.069872,COMPLETE
1,1,0.0,2023-11-03 20:44:43.809105,2023-11-03 20:47:32.623467,0 days 00:02:48.814362,7,29,23,5,66,0.905519,0.695855,0.598863,4,0.136507,COMPLETE
2,2,0.0,2023-11-03 20:47:32.625968,2023-11-03 20:51:33.050330,0 days 00:04:00.424362,18,16,16,14,111,0.868586,0.645769,0.743613,6,0.127380,COMPLETE


In [15]:
from IPython.display import Image

fig1 = optuna.visualization.plot_slice(study)
Image(fig1.to_image(format="png"))

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
fig2 = optuna.visualization.plot_param_importances(study)
Image(fig2.to_image(format="png"))

In [17]:
best_model = Sequential()
best_model.add(Conv2D(best_params['conv1_units'], kernel_size=(3, 3), activation='relu', input_shape=(240, 240, 3)))
best_model.add(Conv2D(best_params['conv2_units'], kernel_size=(3, 3), activation='relu'))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Dropout(best_params['dropout1']))
best_model.add(Conv2D(best_params['conv3_units'], kernel_size=(3, 3), activation='relu'))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Conv2D(best_params['conv4_units'], kernel_size=(3, 3), activation='relu'))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Dropout(best_params['dropout2']))
best_model.add(Flatten())
best_model.add(Dense(best_params['dense_units'], activation='relu'))
best_model.add(Dropout(best_params['dropout3']))
best_model.add(Dense(9, activation='softmax'))

best_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

num_epochs = 10
history = train_and_record_accuracy(best_model, X_train, y_train, X_test, y_test, num_epochs)

train_accuracy_values = history["train_accuracy"]
val_accuracy_values = history["val_accuracy"]

10/10 [==============================] - 12s 1s/step - loss: 2.4303 - accuracy: 0.1472 - val_loss: 2.2134 - val_accuracy: 0.0000e+00
Epoch 1: Train Accuracy = 0.1878, Validation Accuracy = 0.0000
10/10 [==============================] - 14s 1s/step - loss: 2.1660 - accuracy: 0.1658 - val_loss: 2.2267 - val_accuracy: 0.0000e+00
Epoch 2: Train Accuracy = 0.2149, Validation Accuracy = 0.0000
10/10 [==============================] - 10s 1s/step - loss: 2.1513 - accuracy: 0.1743 - val_loss: 2.2433 - val_accuracy: 0.0000e+00
Epoch 3: Train Accuracy = 0.2657, Validation Accuracy = 0.0000
10/10 [==============================] - 10s 1s/step - loss: 2.1415 - accuracy: 0.1506 - val_loss: 2.2497 - val_accuracy: 0.0000e+00
Epoch 4: Train Accuracy = 0.1946, Validation Accuracy = 0.0000
10/10 [==============================] - 10s 1s/step - loss: 2.1524 - accuracy: 0.1624 - val_loss: 2.2760 - val_accuracy: 0.0000e+00
Epoch 5: Train Accuracy = 0.2166, Validation Accuracy = 0.0000
10/10 [=============

In [19]:
import pickle

with open("best_model.pkl", 'wb') as file:
    pickle.dump(best_model, file)

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(train_accuracy_values) + 1)

# Plot the training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_accuracy_values, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy_values, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(False)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Display the plot
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
study.best_params

{'kernel_size': 8,
 'conv1_units': 20,
 'conv2_units': 25,
 'dropout1': 0.9242546115857789,
 'conv3_units': 8,
 'conv4_units': 6,
 'dropout2': 0.6887097227436749,
 'dense_units': 125,
 'dropout3': 0.6843400442253731,
 'learning_rate': 0.09215853447702334}